In [1]:
import os
import open3d as o3d
import numpy as np
from PIL import Image
from tqdm import tqdm
import random
import pdb
from pdb import set_trace as bp

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def read_obj(obj_path):
    with open(obj_path) as file:
        points = []
        while 1:
            line = file.readline()
            if not line:
                break
            strs = line.split(" ")
            if strs[0] =='v':
                points.append((float(strs[1]),float(strs[2]),float(strs[3])))

    points = np.array(points)

    return points

In [3]:
def change(pos_array):
    p = pos_array
    p_new = np.zeros(p.shape)
    for i in range(len(p)):
        p_new[i] = np.array([p[i,0],-p[i,2],-p[i,1]])
    return p_new

In [4]:
if not os.path.exists('./traj'):
    os.makedirs('./traj')

In [5]:
file_dir = 'F:/mesh_file8'
file_number = len(os.listdir(file_dir))
animal_name = file_dir.split('/')[-1]

In [6]:
# pic = os.path.join(file_dir,'{:04d}.png'.format(1))
# H,W = np.array(Image.open(pic)).shape[:-1]

In [7]:
point_array = np.array([])
for i in tqdm(range(file_number)):
    points = change(read_obj(os.path.join(file_dir,'Frame{}.obj'.format(i+1))))
    points = points.reshape(1,-1,3)
#     point_list.append(points)
    if len(point_array)==0:
        point_array = points
    else:
        point_array = np.concatenate((point_array,points),axis=0)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 56.27it/s]


In [8]:
select_index = np.array(random.sample(range(point_array.shape[1]), 100))

In [9]:
for i in tqdm(range(file_number)):
    previous = max(0,i+1-15)
    tmp  = point_array[previous:(i+1),select_index]
    # tmp[:, 1] *= -1
    np.save('./traj/{}.npy'.format(i+1),tmp)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 992.75it/s]


In [10]:
color_array = np.random.random((40,3))
W1 = np.load(os.path.join('F:/GOOGLEDOWNLOAD','W1 (3).npy'))
# (W1@color_array).shape
color_matrix = W1@color_array

In [11]:
W = H = 1024

In [12]:
info_index = 2
line_info = np.load('traj/{}.npy'.format(info_index))
line_info.shape

(2, 100, 3)

In [13]:
def test_show(info_index):
    
    line_info = np.load('traj/{}.npy'.format(info_index))
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=W,height=H)
    
    ctr = vis.get_view_control()
    
    mesh = o3d.io.read_triangle_mesh(os.path.join(file_dir,'Frame{}.obj'.format(info_index)))
    
    ########## Transformation ##########
    points = np.array(mesh.vertices)
    points = points[:, [0, 2, 1]]
    points[:, 1] *= -1
    mesh.vertices = o3d.utility.Vector3dVector(points)
    ####################################
    
#     mesh.paint_uniform_color([1, 0.706, 0])
    mesh.vertex_colors = o3d.utility.Vector3dVector(W1@color_array)
    vis.add_geometry(mesh)
    vis.update_geometry(mesh)
    for index in range(line_info.shape[1]):
        points = line_info[:,index]
        lines = np.array([[i,i+1] for i in range(len(points)-1)])
        points[:, 2] *= -1
#         colors = [[1, 0, 0] for i in range(len(lines))]
        colors = [color_matrix[select_index[index]] for i in range(len(lines))]
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
    
        vis.add_geometry(line_set)
        vis.update_geometry(line_set)
    
    information = np.load(os.path.join('F:/info','{:04d}.npz'.format(info_index)))
    intrin = information['intrinsic_mat']
    extrin = np.concatenate( (information['extrinsic_mat'],np.array([0,0,0,1]).reshape((1,4))))
                              
    init_param = ctr.convert_to_pinhole_camera_parameters()
    w, h = W,H
    fx = intrin[0,0]
    fy = intrin[1,1]
    cx = intrin[0,2]
    cy = intrin[1,2]
    init_param.intrinsic.width = w
    init_param.intrinsic.height = h
    
    init_param.intrinsic.set_intrinsics(init_param.intrinsic.width, init_param.intrinsic.height, fx, fy, cx, cy)
    init_param.extrinsic = extrin
    ctr.convert_from_pinhole_camera_parameters(init_param)
    
#     vis.poll_events()
#     vis.update_renderer()
    
    image = vis.capture_screen_float_buffer(do_render=True)

    ####
    # vis.run()
    ####
    # vis.destroy_window()
    
    image = np.array(image) * 255
    image = image.astype(np.uint8)
    
    return image


In [14]:
for i in tqdm(range(file_number)):
    image = test_show(i+1)
    img = Image.fromarray(image)
    img.save('D:/imagerepo4/{}.png'.format(i+1))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or bindi

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:49<00:00,  2.04it/s]


In [111]:
import numpy as np

In [113]:
occ = np.random.randint

In [114]:
occ.shape

(3,)

In [17]:
W1.sum(1)

array([1.        , 1.        , 0.9999999 , ..., 0.9999999 , 0.99999994,
       1.        ], dtype=float32)